## Deteccion del tablero en la imagen

-----
Se usara el tablero B para esta seccion.

### Representaciones:
<pre>
Se usa una representacion de tablero mediante un array de 64 elementos de 8 bits cada uno. En cada uno de esos elementos se
 almacenara una pieza. Aqui se presentan los encodings para cada una de ellas. 
    * Los primeros 4 bits se usaran para representar el color.
        - 0000 XXXX : Negro
        - 1111 XXXX : Blanco
    * Con los 4 bits restantes se representa la pieza:
         Pieza        Codificacion
        - Peon      | 0000
        - Caballo   | 0001
        - Alfil     | 0010
        - Torre     | 0011
        - Reina     | 0100
        - Rey       | 0101
    * Un casillero vacio se representa mediante: 1111 1111
</pre>
----------------------------------------
### Detalle del algoritmo:
<pre>
El algoritmo consta de 4 partes:
        
    1. Deteccion del tablero: Cuando la foto se captura el tablero puede estar torcido y puesto en cualquier tipo de posicion. Por 
    lo que lo primero es identificarlo, recortarlo, centrarlo y alinearlo.

    2. Luego se reconocen cada una de las casillas individuales. Se determinan sus dimensiones, mediante 2 puntos, la esquina 
    superior izquierda y la esquina inferior izquierda.

    3. Se aplica 2 mascaras. Una de ellas detecta piezas blancas, la otra negras. Si se detecta una pieza se almacena en un array de 
    64 elementos booleanos.

    4. Se compara el array de booleanos con el de piezas y se realizan las debidas actualizaciones.  

A tener en cuenta: 

Para el correcto funcionamiento del algoritmo se debe conocer la posicion inicial de las piezas, ya que en base a estas se realiza 
la deduccion de posiciones despues.
</pre>
--------
#### INVESTIGAR:
* formatos de compresion de la jpeg.
* compresion con perdida y sin perdida.
* investigar el "blur" para aplicar a las imagenes previa a la resta. - blur gausseano 


In [2]:
import cv2
import numpy as np

In [27]:
# Cargo las imagenes del tablero b
b_pos_inicio = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/pos-inicial.jpeg")
b_pos_inicio2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/pos-inicial-2.jpeg")
tablerob_mov1 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-1.jpg")
tablerob_mov2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-2.jpg")
tablerob_mov3 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-3.jpg")
tablerob_mov4 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-4.jpg")
tablerob_mov5 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-5.jpg")
tablerob_mov6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-6.jpg")

# Cargo imagenes del tablero sonar
sonar_1 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-2-1.jpg")
sonar_2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-2-2.jpg")
sonar_3 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-2-3.jpg")
sonar_4 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-3-1.jpg")
sonar_5 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-3-2.jpg")
sonar_6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-3-4.jpg")
sonar_6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-4.jpg")
sonar_6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-6.jpg")
sonar_6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-7.jpg")
sonar_6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-8.jpg")
sonar_6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/sonar-10.jpg")

#Cargo imagenes ideales
ideal = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/workable.jpg")
ideal2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/workable2.jpg")

In [1]:
# Funciones comunes 

def showImg(img, text='image'):
    cv2.namedWindow(text, cv2.WINDOW_KEEPRATIO)
    cv2.imshow(text, img)
    cv2.resizeWindow(text, 700, 700)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
def mostrarContornos(img,min,max, bool):
    # Aplicar deteccion de bordes utilizando Canny
    edges = cv2.Canny(img, min, max, apertureSize=3)
    if bool:
        showImg(edges, 'Detector de contornos')
    return edges

# 1. Deteccion y recorte del tablero

In [4]:
# Trabajaremos con esta posicion.
showImg(b_pos_inicio, "Posicion inicial")

#### Modelo final

In [6]:
import cv2
import numpy as np

def showImg(img, text='image'):
    cv2.namedWindow(text, cv2.WINDOW_KEEPRATIO)
    cv2.imshow(text, img)
    cv2.resizeWindow(text, 700, 700)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
def mostrarContornos(img,min,max, bool):
    # Aplicar deteccion de bordes utilizando Canny
    edges = cv2.Canny(img, min, max, apertureSize=3)
    if bool:
        showImg(edges, 'Detector de contornos')
    return edges

def rotarRecortarImagen(img, verbose=False):
    # Convertimos la imagen a gris
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    if (verbose): showImg(img_gray, 'Imagen en blanco y negro')

    # Aplicar deteccion de bordes utilizando Canny
    edges = mostrarContornos(img_gray, 150, 300, verbose)

    if (verbose): showImg(edges, 'Contornos detectados')

    # Encontrar las lineas presentes en la imagen utilizando la transformada de Hough
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

    # Calcular el angulo promedio solo de las líneas horizontales detectadas
    angle = np.mean([np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) for line in lines if abs(line[0][3] - line[0][1]) < abs(line[0][2] - line[0][0])])

    # Rotar la imagen utilizando el ángulo calculado
    (h, w) = img_gray.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle * 180 / np.pi, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    if (verbose): showImg(rotated, 'Imagen rotada')

    edges = mostrarContornos(rotated, 150, 300, verbose)

    #  Convierto la imagen a hsv
    #hsv_image = cv2.cvtColor(rotated, cv2.COLOR_BGR2HSV)


    #if (verbose): showImg(hsv_image, 'HSV aplicado a la imagen rotada')

    # Defino el rango de colores para poder recortar el tablero, en este caso se usa teniendo en cuenta el fondo de color blanco
    #lower_color = np.array([0, 0, 0])
    #upper_color = np.array([255, 90, 200])

    # Creo una mascara que identifique aproximadamente el color #778078, este es el color del fondo.
    #mask = cv2.inRange(rotated, lower_color, upper_color)

    if (verbose): showImg(edges, 'Mascara, hsv, imagen rotada')

    # Se encuentran los contornos
    contorno,_ = cv2.findContours(edges,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cont = 0

    booleano = True

    for c in contorno:
        area = cv2.contourArea(c)
        if (area > 10000):
            cont = cont + 1
            x,y,w,h = cv2.boundingRect(c)
            # Recortamos el contorno encontrado.
            if (booleano):
                # rotamos la imagen encontrada
                cropped_image = rotated[(y+10):(y+h-5), (x+10):(x+w-5)].copy()
                if (verbose): showImg(cropped_image, "Imagen recortada")
                booleano = False
            area = cv2.contourArea(c)
    
    if (verbose): showImg(cropped_image, 'Imagen rotada y recortada')

    return cropped_image

def recortarCasillero(image, i, j):
    width = image.shape[1]
    height = image.shape[0]
    start_y = int((height/8)*i)
    end_y = int((height/8)*i+(height/8))
    start_x = int((width/8)*j)
    end_x = int((width/8)*j+(width/8))
    casillero = image[start_y:end_y, start_x:end_x].copy()
    return casillero

def encontrarContornosPieza(image, mask, area_max=6000, area_min=200):
    retorno = image.copy()
    contorno,_ = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    grid_contours = []   
    for contour in contorno:
        if area_max > cv2.contourArea(contour) > area_min:
            grid_contours.append(contour)

    cv2.drawContours(retorno, grid_contours, -1, (0, 255, 0), 2)
    return retorno, len(grid_contours)

# casilla1 : imagen recortada del casillero 
# verbose : imprime las imagenes de los casilleros con los filtros aplicados y contornos detectados
# white : es un booleano, true para detectar piezas rojas, false para piezas negras
def detectarPieza(casilla1, verbose=False):
    lower_color_black = np.array([0, 0, 0])
    upper_color_black = np.array([90, 90, 90])
    mascara1 = cv2.inRange(casilla1, lower_color_black, upper_color_black).copy()
    img, contorno = encontrarContornosPieza(casilla1,mascara1,6000, 200)
    if verbose:
        showImg(casilla1)
        showImg(img, 'Contornos')
    if contorno > 0:
        if verbose: print('Pieza detectada')
        return 'b'
    else:
        lower_red = np.array([0, 100, 100])
        upper_red = np.array([255, 255, 255])

        mascara1 = cv2.inRange(casilla1, lower_red, upper_red).copy()
        img, contorno = encontrarContornosPieza(casilla1,~mascara1,6000, 100)

        if verbose:
            showImg(casilla1)
            showImg(img, 'Contornos')
        if contorno > 0:
            if verbose: print('Pieza detectada')
            return 'r'
        else:
            if verbose: print('Casillero vacio')
            return '.'    
        
image = rotarRecortarImagen(ideal2)

matriz = np.empty((8, 8), dtype=str)
for i in range(8):
    for j in range(8):
        cropped_image = recortarCasillero(image, i,j)
        matriz[i][j] = detectarPieza(cropped_image, False)
print(matriz)  



NameError: name 'ideal2' is not defined

In [6]:
cropped_image = recortarCasillero(image, 4,2)

lower_red = np.array([0, 80, 80])
upper_red = np.array([255, 255, 255])

mascara1 = cv2.inRange(cropped_image, lower_red, upper_red).copy()
img, contorno = encontrarContornosPieza(cropped_image,~mascara1,6000, 100)

print(contorno)
showImg(img)

5


In [5]:
import cv2
import numpy as np

def shadow_removal(image):
  """Removes shadows from an image using OpenCV.

  Args:
    image: The image to remove shadows from.

  Returns:
    The image with shadows removed.
  """

  # Convert the image to the YCbCr color space.
  ycbcr = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

  # Calculate the shadow mask.
  shadow_mask = np.where((ycbcr[:, :, 0] - ycbcr[:, :, 2]) > 30, 255, 0)

  shadow_mask = shadow_mask.astype("uint8")

  # Remove the shadows from the image.
  shadowless_image = cv2.bitwise_and(image, image, mask=shadow_mask)



  return shadowless_image

image = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tom.jpg")
# Remove the shadows from the image.
shadowless_image = shadow_removal(image)
# Display the original and shadowless images.
showImg(image, "Original")
showImg(shadowless_image, "Shadowless")


In [11]:
def dibujarContornos(img, masked_img, title, bool):
    retorno = img.copy()
    edges = cv2.Canny(masked_img, 200, 500).copy()
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    chessboard_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if (area > 1000):
            chessboard_contours.append(contour)
    for contour in chessboard_contours:
        cv2.drawContours(retorno , [contour], -1, (0, 255, 0), 2).copy()
    if bool:
        showImg(retorno , title)
    return retorno 

-------
### Deteccion de piezas en casilleros.

In [12]:
# mascara para casillero blanco pieza blanca
lower_cblanco_pblanco = np.array([100,100,50],np.uint8)
upper_cblanco_pblanco = np.array([170,150,150],np.uint8)

# mascara para casillero negro con pieza negra
lower_cnegro_pnegra = np.array([30,30,30],np.uint8)
upper_cnegro_pnegra = np.array([100,50, 50],np.uint8)

# mascara para casillero negro con pieza blanca
lower_cnegro_pblanca = np.array([0, 0, 0])
upper_cnegro_pblanca = np.array([50, 50, 50])

# mascara para casillero blanco con pieza negra
lower_cblanco_pnegra = np.array([0, 0, 0])
upper_cblanco_pnegra = np.array([50, 50, 50])

lower_mask1 = np.array([100,100,50],np.uint8)
upper_mask1 = np.array([170,150,150],np.uint8)

lower_mask2 = np.array([30,30,30],np.uint8)
upper_mask2 = np.array([100,50, 50],np.uint8)

lower_mask3 = np.array([0, 0, 0])
upper_mask3 = np.array([50, 50, 50])

In [20]:
def detectarPieza(casilla1, verbose=False):
    mascara1 = cv2.inRange(casilla1, lower_mask1, upper_mask1).copy()
    mascara2 = cv2.inRange(casilla1, lower_mask2, upper_mask2).copy()
    mascara3 = cv2.inRange(casilla1, lower_mask3, upper_mask3).copy()
    
    img1, contorno1 = encontrarContornosPieza(casilla1,mascara1)
    img2, contorno2 = encontrarContornosPieza(casilla1,mascara2)
    img3, contorno3 = encontrarContornosPieza(casilla1,mascara3)
    if verbose:
        showImg(casilla1)
        showImg(img1, 'img1')
        showImg(img2, 'img2')
        showImg(img3, 'img3')
        print(contorno1,contorno2,contorno3)
    if contorno1 > 0 and contorno2 > 0 and contorno3 > 0:
        if verbose: print('Pieza detectada')
        return '1'
    else:
        if verbose: print('Casillero vacio')
        return '.'        

def recortarCasillero(image, i, j):
    start_y = int(100*i)
    end_y = int(97*i+85)
    start_x = int(97*j)
    end_x = int(97*j+85)
    casillero = image[start_y:end_y, start_x:end_x].copy()
    return casillero


In [25]:
image = rotarRecortarImagen(sonar_5, True) 

#Procesar todo el tablero:
matriz = np.empty((8, 8), dtype=str)
for i in range(8):
    for j in range(8):
        cropped_image = recortarCasillero(image, i,j)
        matriz[i][j] = detectarPieza(cropped_image,False)
print(matriz)     

showImg(image)

[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '1']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]


In [10]:
# TODO: Ver como se puede utilizar la informacion del heatmap, esta interesante ...
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv_image)
heatmap = cv2.applyColorMap(v, cv2.COLORMAP_HSV)

showImg(heatmap, 'heatmap')

-----------------------
# Tablero B

In [11]:
 # Carga las imagenes en una variable
a_pos_inicio = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-inicio.jpeg")
pos_mid = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid.jpeg")
pos_mid2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid2.jpeg")
pos_ataque = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-ataque.jpeg")
iluminada = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/iluminada.jpeg")
pos_inicial_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-inicio-esp.jpeg")
pos_mid_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid-esp.jpeg")
pos_mid2_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid2-esp.jpeg")
tablero_normal = cv2.imread('/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-ideal/tablero-comun.png')

In [12]:
# procesamos la imagen 
cv2.imshow('Pos mid', pos_mid)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Convert the image to grayscale
pos_mid_gray = cv2.cvtColor(pos_mid, cv2.COLOR_BGR2GRAY)

cv2.imshow('Pos mid grayscale', pos_mid_gray)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Find the chessboard corners
cv2.imshow('tablero normal: ',tablero_normal)
cv2.waitKey(10000)
cv2.destroyAllWindows()

ret, corners = cv2.findChessboardCorners(pos_mid_gray, (9, 9))

print(ret, corners)

# Draw the chessboard corners on the image
if ret:
  cv2.drawChessboardCorners(pos_mid, (9, 9), corners, ret)

# Display the image
cv2.imshow('edges detected', pos_mid) 
cv2.waitKey(5000)
cv2.destroyAllWindows()

False None


In [13]:
#import cv2
#import numpy as np
#
#def nothing(x):
#    pass
#
## Create a black image, a window
#img = np.zeros((300, 512, 3), np.uint8)
#cv2.namedWindow('image')
#
## create trackbars for color change
#cv2.createTrackbar('R', 'image', 0, 255, nothing)
#cv2.createTrackbar('G', 'image', 0, 255, nothing)
#cv2.createTrackbar('B', 'image', 0, 255, nothing)
## create switch for ON/OFF functionality
#switch = '0 : OFF \n1 : ON'
#cv2.createTrackbar(switch, 'image', 0, 1, nothing)
#while (1):
#    cv2.imshow('image', img)
#    k = cv2.waitKey(1) & 0xFF
#    if k == 27:
#        break
#    # get current positions of four trackbars
#    r = cv2.getTrackbarPos('R', 'image')
#    g = cv2.getTrackbarPos('G', 'image')
#    b = cv2.getTrackbarPos('B', 'image')
#    s = cv2.getTrackbarPos(switch, 'image')
#    if s == 0:
#        img[:] = 0
#    else:
#        img[:] = [b, g, r]
#cv2.destroyAllWindows()
#

# Resta de tableros

ahora viene lo divertido ...    

In [35]:

import numpy as np

m0 = np.array([['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r'],
     ['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r'],
     ['.', '.', '.', '.', '.', '.', '.', '.'],
     ['.', '.', '.', '.', '.', '.', '.', '.'],
     ['.', '.', '.', '.', '.', '.', '.', '.'],
     ['.', '.', '.', '.', '.', '.', '.', '.'],
     ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b'],
     ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b']])


m1 = [['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r'],
     ['r', 'r', 'r', '.', 'r', 'r', 'r', 'r'],
     ['.', '.', '.', '.', '.', '.', '.', '.'],
     ['.', '.', '.', 'r', '.', '.', '.', '.'],
     ['.', '.', '.', '.', '.', '.', '.', '.'],
     ['.', '.', '.', '.', '.', '.', '.', '.'],
     ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b'],
     ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b']]

m2 = [['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r'],
     ['r', 'r', 'r', '.', 'r', 'r', 'r', 'r'],
     ['.', '.', '.', '.', '.', '.', '.', '.'],
     ['.', '.', '.', 'r', '.', '.', '.', '.'],
     ['.', '.', '.', '.', 'b', '.', '.', '.'],
     ['.', '.', '.', '.', '.', '.', '.', '.'],
     ['b', 'b', 'b', 'b', '.', 'b', 'b', 'b'],
     ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b']]

m0_n = np.array([[2, 2, 2, 2, 2, 2, 2, 2],
                 [2, 2, 2, 2, 2, 2, 2, 2],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1]])

m1_mov_blanco = np.array([[2, 2, 2, 2, 2, 2, 2, 2],
                 [2, 2, 2, 0, 2, 2, 2, 2],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 2, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [1, 1, 1, 1, 1, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1]])

m2_mov_negro = np.array([[2, 2, 2, 2, 2, 2, 2, 2],
                 [2, 2, 2, 0, 2, 2, 2, 2],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 2, 0, 0, 0, 0],
                 [0, 0, 0, 0, 1, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [1, 1, 1, 1, 0, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1]])

m3_blancocomenegro = np.array([[2, 2, 2, 2, 2, 2, 2, 2],
                 [2, 2, 2, 0, 2, 2, 2, 2],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 2, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [1, 1, 1, 1, 0, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1]])

m4_negrocomeblanco = np.array([[2, 2, 2, 2, 2, 2, 2, 2],
                 [2, 2, 2, 0, 2, 2, 2, 2],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 1, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [1, 1, 1, 1, 0, 1, 1, 1],
                 [1, 1, 1, 1, 1, 1, 1, 1]])

#print(res)
#print(puntos)

matriz_mov_tipos = np.array([[1,-1],  # mov negro 0 
                            [2,-1],  # pieza blanca come pieza negra 1 
                            [1,1],   # pieza negra come pieza blanca 2
                            [2,-2]])  # mov blanco 3 

def tienen_mismos_numeros(array1, array2):
    conjunto1 = set(array1)
    conjunto2 = set(array2)
    return conjunto1 == conjunto2

def determinarPuntos(mov1, mov2):
    res = mov1 - mov2

    # puntos =  [res, x, y]

    puntos = np.empty((2,3), dtype=int)
    cont = 0

    for i in range(2):
        for j in range(3):
            if (res[i][j] != 0):
                 puntos[cont][0] = res[i][j] 
                 puntos[cont][1] = i
                 puntos[cont][2] = j
                 cont = cont + 1

    diferencias = np.array([puntos[0][0], puntos[1][0]])

    caso = -1

    for i in range(0,4):
        if (tienen_mismos_numeros(diferencias, matriz_mov_tipos[i])):
            caso = i
            break

    origen =  np.empty((2), dtype=int)
    destino = np.empty((2), dtype=int)

    if (caso == 0):
        for i in range(2):
            if (puntos[i][0] == 1): 
                origen[0] = puntos[i][1]
                origen[1] = puntos[i][2]
            else:
                destino[0] = puntos[i][1]
                destino[1] = puntos[i][2]
    elif (caso == 1):
        for i in range(2):
            if (puntos[i][0] == 2): 
                origen[0] = puntos[i][1]
                origen[1] = puntos[i][2]
            else:
                destino[0] = puntos[i][1]
                destino[1] = puntos[i][2]
    elif (caso == 2):
        for i in range(2):
            if (mov2[i][0] == 2): 
                origen[0] = puntos[i][1]
                origen[1] = puntos[i][2]
            else:
                destino[0] = puntos[i][1]
                destino[1] = puntos[i][2]
    elif (caso == 3):
        for i in range(2):
            if (puntos[i][0] == 2): 
                origen[0] = puntos[i][1]
                origen[1] = puntos[i][2]
            else:
                destino[0] = puntos[i][1]
                destino[1] = puntos[i][2]
    return origen, destino




print(origen)

print(destino)


[1 3]
[3 3]


In [7]:
print(m1_mov_blanco - m2_mov_negro)

[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0]]


In [8]:
print(m2_mov_negro - m3_blancocomenegro)

[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]


In [9]:
print(m2_mov_negro - m4_negrocomeblanco)

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
